# Setting Up

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import os
import torch.utils.data as data
import random
import shutil
from torch.optim.lr_scheduler import StepLR

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Please set GPU via Edit -> Notebook Settings.")
    DEVICE = torch.device("cpu")


# Define some common variables for dtypes/devices.
# These can be keyword arguments while defining new tensors.
to_float = {"dtype": torch.float32, "device": DEVICE}
to_double = {"dtype": torch.float64, "device": DEVICE}

Good to go!


In [ ]:
# Put any Preprocessing steps here

# Create DataLoader

In [ ]:
# DATALOADER V2
def create_dataset(window, csv_folder):
    """Transform a time series into a prediction dataset

    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []

    data = []
    # minlength = 100
    droplastrows = 8
    rowsperfile = 44 - droplastrows
    for file in os.listdir(csv_folder):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(csv_folder, file))
            df.drop(df.tail(1).index,inplace=True)
            df.drop(df.head(1).index,inplace=True)
            df.replace('<not counted>', np.nan, inplace=True)
            df = df.dropna(axis=1, thresh=int(0.5 * len(df)))
            df = df.dropna(axis=0, how='any')
            df = df.drop(df.tail(8).index)
            rows = []
            df = df[['L1-icache-load-misses', 'L1-icache-loads']] # FEATURE SELECTION
            # df = df[['L1-icache-loads']] # FEATURE SELECTION
            # df = df[['L1-icache-load-misses']] # FEATURE SELECTION

            for index, row in df.iterrows():
              rows.append((row.values).astype(int))
            # minlength = min(minlength, len(df))
            # self.data.append(rows[:self.rowsperfile])
            # print(len(rows[:rowsperfile]), len(rows[0]))
            for i in range(rowsperfile - window):
                feature = rows[i:i+window]
                target = rows[i+window]
                # print(len(feature))
                # print(len(target))
                X.append(feature)
                y.append(target)
    # print(minlength)

    # print(len(X))
    # print(len(y))
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [ ]:
def create_eval_dataset(window, csv_folder):
    """Transform a time series into a prediction dataset

    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    data = []
    droplastrows = 8
    rowsperfile = 44 - droplastrows
    for file in os.listdir(csv_folder):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(csv_folder, file))
            df.drop(df.tail(1).index,inplace=True)
            df.drop(df.head(1).index,inplace=True)
            df.replace('<not counted>', np.nan, inplace=True)
            df = df.dropna(axis=1, thresh=int(0.5 * len(df)))
            df = df.dropna(axis=0, how='any')
            rows = []
            # df = df[['L1-icache-loads']] # FEATURE SELECTION
            df = df[['L1-icache-load-misses', 'L1-icache-loads']] # FEATURE SELECTION
            # df = df[['L1-icache-load-misses']] # FEATURE SELECTION

            for index, row in df.iterrows():
              rows.append((row.values).astype(int))
            Xf = []
            yf = []
            for i in range(rowsperfile - window):
                feature = rows[i:i+window]
                target = rows[i+window]
                Xf.append(feature)
                yf.append(target)
            X.append(Xf)
            y.append(yf)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Create Model

In [ ]:
class RNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=2, hidden_size=40, num_layers=1, batch_first=True)
        self.linear = nn.Linear(40, 2)
        self.init_weights(self.linear)
        self.init_weights(self.lstm)

    def forward(self, x):

        batch = x.shape[0]
        sequence = x.shape[1]
        cols = x.shape[2]

        x, _ = self.lstm(x)
        x = x[:, -1, :]

        x = self.linear(x)
        return x


    def init_weights(self, layer):
        for name, param in layer.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)

In [ ]:
path = '/content/drive/MyDrive/573project/573_LSTM/data2'
dir_list = os.listdir(path)

print("Files and directories in '", path, "' :")

# prints all files
print(dir_list)

Files and directories in ' /content/drive/MyDrive/573project/573_LSTM/data2 ' :
['train', 'test', 'train2', 'final']


# Initialize Hyperparamters

In [ ]:
# Define hyperparameters
input_size = 2
hidden_size = 64
num_layers = 1
output_size = 2
sequence_length = 20
batch_size = 128
learning_rate = 1e-4
num_epochs = 120
data_folder = path
train_path = data_folder + '/train2'
test_path = data_folder + '/test'

# model 3, hidden layer = 40, features: cache misses, learning rate 1e-1, epochs 300, scheduler 0.1 at 200, sequence length 30, don't normalize
# model 4, hidden layer = 40, features: cache misses, learning rate 1e-4, epochs 120, sequence length 24, drop the last 8 rows, normalized
# model 5, hidden layer = 40, features: cache misses, learning rate 1e-4, epochs 120, sequence length 20, drop the last 8 rows, normalized

# Randomize some of the training data and split datasets

In [ ]:
# def split_files(source_folder, folder1, folder2, split_ratio=0.88):
#     # Get a list of all CSV files in the source folder
#     csv_files = [file for file in os.listdir(source_folder) if file.endswith(".csv")]

#     # Calculate the number of files for each folder based on the split ratio
#     num_files_folder1 = int(len(csv_files) * split_ratio)
#     num_files_folder2 = len(csv_files) - num_files_folder1

#     # Randomly shuffle the list of CSV files
#     random.shuffle(csv_files)

#     # Split the files into two lists based on the calculated counts
#     files_folder1 = csv_files[:num_files_folder1]
#     files_folder2 = csv_files[num_files_folder1:]

#     # Copy files to folder1
#     for file in files_folder1:
#         source_path = os.path.join(source_folder, file)
#         destination_path = os.path.join(folder1, file)
#         shutil.copy2(source_path, destination_path)

#     # Copy files to folder2
#     for file in files_folder2:
#         source_path = os.path.join(source_folder, file)
#         destination_path = os.path.join(folder2, file)
#         shutil.copy2(source_path, destination_path)

#     print(f"Number of files in {folder1}: {len(files_folder1)}")
#     print(f"Number of files in {folder2}: {len(files_folder2)}")

# # Example usage
# source_folder = data_folder + '/train'
# folder1 = data_folder + '/train2'
# folder2 = data_folder + '/final'

# split_files(source_folder, folder1, folder2, split_ratio=0.88)

Number of files in /content/drive/MyDrive/573project/573_LSTM/data2/train2: 565
Number of files in /content/drive/MyDrive/573project/573_LSTM/data2/final: 78


#Initialize Loader

In [ ]:
# Create DataLoader V2
X_train, y_train = create_dataset(sequence_length, train_path)
X_test, y_test = create_dataset(sequence_length, test_path)
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batch_size, drop_last=True)

print(X_train.shape)


<ipython-input-5-bc77fe16ec0c>:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


torch.Size([9040, 20, 2])


In [ ]:
# # Normalize DataLoader - ONLY RUN ONCE, comment out max1 values if only using 1 feature, comment out both if not normalizing

Xtrain_max0 = torch.max(X_train[:,:,0]) # maximum column value between each data split
Xtrain_max1 = torch.max(X_train[:,:,1])

ytrain_max0 = torch.max(y_train[:,0]) # maximum column value between each data split
ytrain_max1 = torch.max(y_train[:,1])


Xtest_max0 = torch.max(X_test[:,:,0])
Xtest_max1 = torch.max(X_test[:,:,1])

ytest_max0 = torch.max(y_test[:,0])
ytest_max1 = torch.max(y_test[:,1])

max0 = max(Xtrain_max0, Xtest_max0, ytrain_max0, ytest_max0)
max1 = max(Xtrain_max1, Xtest_max1, ytrain_max1, ytest_max1)


X_train[:,:,0] /= max0
X_train[:,:,1] /= max1

X_test[:,:,0] /= max0
X_test[:,:,1] /= max1

y_train[:,0] /= max0
y_train[:,1] /= max1

y_test[:,0] /= max0
y_test[:,1] /= max1


#Initialize Model and Start Training

In [ ]:
# Create RNN model
rnn = RNNModel()

In [ ]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=200, gamma=0.1)

# # Set patience and other hyperparameters
# patience = 20
# best_test_rmse = float('inf')
# counter = 0

for epoch in range(num_epochs):
    rnn.train()
    for X_batch, y_batch in loader:
        y_pred = rnn(X_batch)
        loss = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()
    rnn.eval()
    with torch.no_grad():
        y_pred_train = rnn(X_train)
        train_rmse = np.sqrt(criterion(y_pred_train, y_train))
        y_pred_test = rnn(X_test)
        test_rmse = np.sqrt(criterion(y_pred_test, y_test))

    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

    # # Check for early stopping
    # if test_rmse < best_test_rmse:
    #     best_test_rmse = test_rmse
    #     counter = 0
    #     torch.save(rnn.state_dict(), 'rnn_model.pth')
    # else:
    #     counter += 1

    # if counter >= patience:
    #     print(f"Early stopping after {epoch} epochs.")
    #     break


# Save the trained model
# torch.save(rnn.state_dict(), 'rnn_model.pth')

Epoch 0: train RMSE 0.2387, test RMSE 0.2440
Epoch 1: train RMSE 0.1905, test RMSE 0.1935
Epoch 2: train RMSE 0.1471, test RMSE 0.1470
Epoch 3: train RMSE 0.1198, test RMSE 0.1192
Epoch 4: train RMSE 0.0974, test RMSE 0.0977
Epoch 5: train RMSE 0.0880, test RMSE 0.0888
Epoch 6: train RMSE 0.0850, test RMSE 0.0860
Epoch 7: train RMSE 0.0834, test RMSE 0.0845
Epoch 8: train RMSE 0.0820, test RMSE 0.0834
Epoch 9: train RMSE 0.0810, test RMSE 0.0827
Epoch 10: train RMSE 0.0802, test RMSE 0.0819
Epoch 11: train RMSE 0.0797, test RMSE 0.0815
Epoch 12: train RMSE 0.0791, test RMSE 0.0810
Epoch 13: train RMSE 0.0789, test RMSE 0.0808
Epoch 14: train RMSE 0.0784, test RMSE 0.0804
Epoch 15: train RMSE 0.0783, test RMSE 0.0803
Epoch 16: train RMSE 0.0780, test RMSE 0.0800
Epoch 17: train RMSE 0.0778, test RMSE 0.0799
Epoch 18: train RMSE 0.0776, test RMSE 0.0798
Epoch 19: train RMSE 0.0775, test RMSE 0.0797
Epoch 20: train RMSE 0.0774, test RMSE 0.0795
Epoch 21: train RMSE 0.0772, test RMSE 0.079

In [ ]:
# Print the parameters of the linear layer
for name, param in rnn.named_parameters():
    if 'linear' in name:
        print(name, param.data)

linear.weight tensor([[-0.1732,  0.2164,  0.1395,  0.3702,  0.1945,  0.2789, -0.0166, -0.1718,
         -0.0074,  0.2269,  0.2882, -0.1131, -0.2905,  0.1939, -0.2095,  0.1330,
          0.0924, -0.1697, -0.0667, -0.2685, -0.3346,  0.4074,  0.1989, -0.3295,
          0.2016, -0.0260,  0.0234, -0.2846, -0.2959, -0.2918,  0.1549, -0.3485,
         -0.0458,  0.2945,  0.1110, -0.3384,  0.4037, -0.4134, -0.2427,  0.1755],
        [-0.0536, -0.1497,  0.3887,  0.0632,  0.0492,  0.2987,  0.3459, -0.0797,
         -0.0744, -0.0258,  0.2252,  0.2337,  0.3164,  0.0460, -0.2775, -0.3190,
         -0.1317, -0.0972,  0.0981,  0.2519,  0.3461,  0.1436, -0.3761,  0.1724,
         -0.0320, -0.2653,  0.1357,  0.2534,  0.1481, -0.2829,  0.3363,  0.0213,
          0.3066, -0.0620, -0.1722, -0.1617,  0.0200,  0.0488, -0.2316,  0.4142]])
linear.bias tensor([-0.0021,  0.0027])


#Anomaly Detection


Inference: Error at timestep t = mean squared error between prediction and ground truth. To detect anomaly, use a D-length decision window to determine if for all timesteps in this window, the error is too high (greater than a specified threshold).

Small decision window length (D) = shorter detection time and less leakage, but higher false positive rate.

Test below.

Need to utilize separate dataset for both vulnerable and non-vulnerable files.

How?

  - Loop through rows in CSV file, and begin detection at certain timestep. Then monitor decision window afterwards.

In [ ]:
dir_list = os.listdir(path + '/final/vuln')

print("Files and directories in '", path + '/final/vuln', "' :")

# prints all files
print(len(dir_list))

Files and directories in ' /content/drive/MyDrive/573project/573_LSTM/data2/final/vuln ' :
799


In [ ]:
# Creating evaluation datasets, comment out normalization step if training set was not normalized
X_vuln_eval, y_vuln_eval = create_eval_dataset(sequence_length, path + '/final/vuln')
X_nonvuln_eval, y_nonvuln_eval = create_eval_dataset(sequence_length, path + '/final/nonvuln')

X_nonvuln_eval[:,:,:,0] /= max0
X_nonvuln_eval[:,:,:,1] /= max1

X_vuln_eval[:,:,:,0] /= max0
X_vuln_eval[:,:,:,1] /= max1

y_nonvuln_eval[:,:,0] /= max0
y_nonvuln_eval[:,:,1] /= max1

y_vuln_eval[:,:,0] /= max0
y_vuln_eval[:,:,1] /= max1

In [ ]:
print(X_train.shape)
# print(X_test.shape)
print(X_nonvuln_eval.shape)
print(y_nonvuln_eval.shape)

# print(X_nonvuln_eval[1,2,:,:])
# print(y_nonvuln_eval[1,1,:])

torch.Size([9040, 20, 2])
torch.Size([78, 16, 20, 2])
torch.Size([78, 16, 2])


In [ ]:
# print((X_nonvuln_eval[1,1,:,:]))
print(X_batch.shape)
# print(X_nonvuln_eval[0,0,:,:].view(1, -1, 1).shape)
print(X_nonvuln_eval[20,0,:,:])
print(rnn(X_nonvuln_eval[20,0,:,:].view(1, -1, 1)))
print(y_nonvuln_eval[20,0,:])

torch.Size([128, 20, 2])
tensor([[0.5163, 0.2184],
        [0.4750, 0.2324],
        [0.5252, 0.1512],
        [0.4969, 0.1423],
        [0.4640, 0.3380],
        [0.4839, 0.1405],
        [0.5075, 0.1396],
        [0.4980, 0.1423],
        [0.5136, 0.1399],
        [0.4078, 0.5212],
        [0.5473, 0.1702],
        [0.4649, 0.1334],
        [0.5296, 0.1453],
        [0.5319, 0.1478],
        [0.5296, 0.1490],
        [0.5305, 0.1477],
        [0.5338, 0.1455],
        [0.5281, 0.1482],
        [0.4668, 0.1320],
        [0.2776, 0.8815]])
tensor([[-0.1037, -0.2674]], grad_fn=<AddmmBackward0>)
tensor([0.5511, 0.1692])


In [ ]:
import torch.nn.functional as F

# Assuming you have two tensors 'tensor1' and 'tensor2' with the same shape
tensor1 = rnn(X_nonvuln_eval[1,1,:,:].view(1, -1, 2))
tensor2 = y_nonvuln_eval[1,1,:].view(1,2)

# Calculate MSE
mse = F.mse_loss(tensor1, tensor2)

print(mse)

tensor(1.7726e-06, grad_fn=<MseLossBackward0>)


In [ ]:
# Anomaly Detection Implementation 1
tn, tp, fn, fp = 0,0,0,0
threshold = 6e-7
percent = 1
# for model 1, use 9e-4 to obtain: True positive:  570 , True negative:  22 , False positive:  56 , False negative:  229
# for model 2, use 2e-4 to obtain: True positive:  544 , True negative:  22 , False positive:  56 , False negative:  255
# for model 3, use 2e2 to obtain: True positive:  487 , True negative:  31 , False positive:  47 , False negative:  312
# normalized model 3, use 9e-6 to obtain: True positive:  525 , True negative:  25 , False positive:  53 , False negative:  274
# normalized model 4, use 7e-7 to obtain: True positive:  536 , True negative:  25 , False positive:  53 , False negative:  263
# normalized model 5, use 1e-7 to obtain: True positive:  550 , True negative:  27 , False positive:  51 , False negative:  249 or True positive:  470 , True negative:  36 , False positive:  42 , False negative:  329
# multivariate model: use 2e-7 to obtain: True positive:  601 , True negative:  16 , False positive:  62 , False negative:  198
print("Non-vulnerable application detection")
avgerror = 0
for i in range(X_nonvuln_eval.shape[0]):
  flags = 0

  for j in range(X_nonvuln_eval.shape[1]):
    error = F.mse_loss(rnn(X_nonvuln_eval[i,j,:,:].view(1, X_nonvuln_eval.shape[2], X_nonvuln_eval.shape[3])), y_nonvuln_eval[i,j,:].view(1,X_nonvuln_eval.shape[3]))
    # print(error, " and " ,threshold)
    # print(error)
    avgerror += error
    if error >  threshold:
      flags += 1
  if flags >= (percent * X_nonvuln_eval.shape[1]):
    fp += 1
  else:
    tn += 1
print(avgerror / (X_nonvuln_eval.shape[0] * X_nonvuln_eval.shape[1]))
print("Vulnerable application detection")
avgerror = 0
for i in range(X_vuln_eval.shape[0]):
  flags = 0
  for j in range(X_vuln_eval.shape[1]):
    error = F.mse_loss(rnn(X_vuln_eval[i,j,:,:].view(1, X_vuln_eval.shape[2], X_nonvuln_eval.shape[3])), y_vuln_eval[i,j,:].view(1,X_nonvuln_eval.shape[3]))
    avgerror += error
    if error > threshold:
      flags += 1
  if flags >= (percent * X_vuln_eval.shape[1]):
    tp += 1
  else:
    fn += 1
    # print(flags)
print(avgerror / (X_vuln_eval.shape[0] * X_vuln_eval.shape[1]))
print("True positive: ", tp, ", True negative: ", tn, ", False positive: ", fp , ", False negative: ", fn)

Non-vulnerable application detection
tensor(0.0050, grad_fn=<DivBackward0>)
Vulnerable application detection
tensor(0.0063, grad_fn=<DivBackward0>)
True positive:  550 , True negative:  26 , False positive:  52 , False negative:  249


# Testing Transformer Model

In [ ]:
import torch
import torch.nn as nn

class TransformerModel(nn.Module):
    def __init__(self):
        super().__init__()
        num_layers = 1
        input_size = 2
        # Replace LSTM with Transformer layer
        self.transformer = nn.Transformer(
            d_model=input_size,
            nhead=1,  # Number of heads in the multiheadattention models
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
        )
        self.linear = nn.Linear(2, 2)

    def forward(self, x):

        x = x.permute(1, 0, 2)
        x = self.transformer(x, x)
        x = x.permute(1, 0, 2)
        x = x[:, -1, :]

        x = self.linear(x)
        return x


In [ ]:
transformer = TransformerModel()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
learning_rate = 0.001
num_epochs = 5

# Loss and optimizer
criterion2 = nn.MSELoss()
optimizer2 = torch.optim.Adam(transformer.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    transformer.train()
    for X_batch, y_batch in loader:
        y_pred = transformer(X_batch)
        loss = criterion2(y_pred, y_batch)
        optimizer2.zero_grad()
        loss.backward()
        optimizer2.step()
    # scheduler.step()
    transformer.eval()
    with torch.no_grad():
        y_pred = transformer(X_train)
        train_rmse = np.sqrt(criterion2(y_pred, y_train))
        y_pred = transformer(X_test)
        test_rmse = np.sqrt(criterion2(y_pred, y_test))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))





# Save the trained model
# torch.save(rnn.state_dict(), 'rnn_model.pth')

Epoch 0: train RMSE 0.1101, test RMSE 0.1138
Epoch 1: train RMSE 0.1102, test RMSE 0.1138
Epoch 2: train RMSE 0.1101, test RMSE 0.1138
Epoch 3: train RMSE 0.1102, test RMSE 0.1138
Epoch 4: train RMSE 0.1101, test RMSE 0.1139


#Transformer Anomaly Detection

In [ ]:
# Anomaly Detection Implementation 1
tn, tp, fn, fp = 0,0,0,0
threshold = 4.5e-4
percent = 1
# with threshold 4e-4, obtain: True positive:  645 , True negative:  17 , False positive:  61 , False negative:  154
# with threshold 5e-4, obtain: True positive:  479 , True negative:  36 , False positive:  42 , False negative:  320
# with threshold 4.5e-4, obtain: True positive:  593 , True negative:  24 , False positive:  54 , False negative:  206
print("Non-vulnerable application detection")
avgerror = 0
for i in range(X_nonvuln_eval.shape[0]):
  flags = 0
  for j in range(X_nonvuln_eval.shape[1]):
    error = F.mse_loss(transformer(X_nonvuln_eval[i,j,:,:].view(1, X_nonvuln_eval.shape[2], X_nonvuln_eval.shape[3])), y_nonvuln_eval[i,j,:].view(1,X_nonvuln_eval.shape[3]))
    # print(error, " and " ,threshold)
    # print(error)
    avgerror += error
    if error >  threshold:
      flags += 1
  if flags >= (percent * X_nonvuln_eval.shape[1]):
    fp += 1
  else:
    tn += 1
print(avgerror / (X_nonvuln_eval.shape[0] * X_nonvuln_eval.shape[1]))
print("Vulnerable application detection")
avgerror = 0
for i in range(X_vuln_eval.shape[0]):
  flags = 0
  for j in range(X_vuln_eval.shape[1]):
    error = F.mse_loss(transformer(X_vuln_eval[i,j,:,:].view(1, X_vuln_eval.shape[2], X_nonvuln_eval.shape[3])), y_vuln_eval[i,j,:].view(1,X_nonvuln_eval.shape[3]))
    avgerror += error
    if error > threshold:
      flags += 1
  if flags >= (percent * X_vuln_eval.shape[1]):
    tp += 1
  else:
    fn += 1
    # print(flags)
print(avgerror / (X_vuln_eval.shape[0] * X_vuln_eval.shape[1]))
print("True positive: ", tp, ", True negative: ", tn, ", False positive: ", fp , ", False negative: ", fn)

Non-vulnerable application detection
tensor(0.0138, grad_fn=<DivBackward0>)
Vulnerable application detection
tensor(0.0139, grad_fn=<DivBackward0>)
True positive:  593 , True negative:  24 , False positive:  54 , False negative:  206


# Try https://unit8co.github.io/darts/generated_api/darts.models.forecasting.dlinear.html and TimeGPT (in-progress)

In [ ]:
from nixtlats import TimeGPT

In [ ]:
timegpt = TimeGPT(token='vXQjOsslTogkzysd8CCD972XqgJhhpH21xeZ2sqnCGTNAk37F5iRC7fYXUDkZQahoOlv3NQb2IH6Lm07y18jjcFLqyDeuzs2wOMkckC9b6wV5X6r5w3ErT76TRvuO6LUxBpZGDVDHWNCPXdbkR6eTQ2Q4vqn7h1EaGgY436SQGe8PdILP0Yy1BUrhOc9cCGZZCz5PJG4IzzxOXov5dzSmOONijnKPifmeinxXI6UAj4JdJVjDU1sARLYpC6Sa2NJ')

In [ ]:
if timegpt.validate_token():
    print("Token validation successful!")  # Token is valid.
else:
    raise Exception("Token validation failed! Please check go to https://dashboard.nixtla.io/ to get your token.")

Token validation successful!


In [ ]:
path = '/content/drive/MyDrive/573project/573_LSTM/data2'
dir_list = os.listdir(path + '/final/nonvuln')

print("Files and directories in '", path, "' :")

# prints all files
print(dir_list)

Files and directories in ' /content/drive/MyDrive/573project/573_LSTM/data2 ' :
['f_rand104.txt.10.stat (3).csv', 'f_rand133.txt.10.stat (2).csv', 'f_rand118.txt.10.stat (2).csv', 'f_rand135.txt.10.stat (3).csv', 'f_rand171.txt.10.stat (2).csv', 'f_rand193.txt.10.stat (2).csv', 'f_rand14.txt.10.stat (2).csv', 'f_rand149.txt.10.stat (3).csv', 'f_rand20.txt.10.stat (2).csv', 'f_rand27.txt.10.stat (3).csv', 'f_rand5.txt.10.stat (3).csv', 'f_rand52.txt.10.stat (3).csv', 'f_rand96.txt.10.stat (3).csv', 'f_rand68.txt.10.stat (2).csv', 'f_rand90.txt.10.stat (3).csv', 'f_rand9.txt.10.stat.csv', 'f_rand97.txt.10.stat (3).csv', 'f_rand81.txt.10.stat (3).csv', 'f_rand78.txt.10.stat (3).csv', 'f_rand93.txt.10.stat (3).csv', 'f_10_rand101.txt.10.stat.csv', 'f_10_rand158.txt.10.stat.csv', 'f_10_rand150.txt.10.stat.csv', 'f_10_rand126.txt.10.stat.csv', 'f_10_rand20.txt.10.stat.csv', 'f_10_rand166.txt.10.stat.csv', 'f_10_rand183.txt.10.stat.csv', 'f_10_rand198.txt.10.stat.csv', 'f_10_rand18.txt.10.sta